In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np
import requests
from tqdm.auto import tqdm

/home/julie/miniconda3/envs/exp-tracking-env/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
model = SentenceTransformer('multi-qa-distilbert-cos-v1')
user_question = "I just discovered the course. Can I still join it?"
v = model.encode(user_question)
v[0], v.dot(v), len(v)

(0.078222625, 1.0000001, 768)

In [3]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [4]:
documents1 = []
for doc in documents:
    if doc['course'] == "machine-learning-zoomcamp": documents1.append(doc)
len(documents1), len(documents)


(375, 948)

In [5]:
ids = [doc['id'] for doc in documents1]


In [5]:
embeddings = []
for doc in documents1:
    question, text = doc['question'], doc['text']
    qa_text = f'{question} {text}'
    embeddings.append(model.encode(qa_text))
X = np.array(embeddings)
X.shape

(375, 768)

In [7]:
scores = X.dot(v)
scores
np.max(scores)

0.65065736

In [8]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents1, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

In [13]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')
ground_truth[0]


{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [14]:
def search_result(query):
    v_query = model.encode(query)
    return search_engine.search(v_query, num_results=5)

In [15]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)


In [16]:
def evaluate(ground_truth, search_function):
    relevance_total = []
    i = 0
    for q in tqdm(ground_truth):
        doc_id = q['document']
        query = q['question']
       # assert doc_id in ids
        
      #  if i==0: print(documents1[ids.index(doc_id)])
        results = search_function(query)
        
        relevance = [doc_id == d['id'] for d in results]
        relevance_total.append(relevance)
        i +=1

    return {
        'hit_rate': hit_rate(relevance_total)
        
    }
#evaluate(ground_truth, search_result)

In [6]:
from elasticsearch import Elasticsearch
#es_client = Elasticsearch('http://localhost:9200')
es_client = Elasticsearch('http://localhost:9200', max_retries=10, retry_on_timeout=True) 
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            
            "text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            
        }     
    }
}




index_name = "ml-course-questions"

In [7]:


es_client.indices.delete(index=index_name, ignore_unavailable=True)


ObjectApiResponse({'acknowledged': True})

In [8]:
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'ml-course-questions'})

In [9]:
for i, doc in enumerate(documents1):
    
    
    doc['text_vector'] = embeddings[i]
    

In [10]:


for doc in tqdm(documents1):
   
    es_client.index(index=index_name, document=doc)

  0%|          | 0/375 [00:00<?, ?it/s]

In [11]:
def elastic_search_knn(field, vector):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs
   

In [12]:

result = elastic_search_knn('text_vector', v)
result

[{'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'},
 {'question': 'I just joined. What should I do next? How can I access course materials?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the vid

In [17]:
def search_result(query):
    v_query = model.encode(query)
    return elastic_search_knn('text_vector', v_query)

evaluate(ground_truth, search_result)


  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.9398907103825137}

In [26]:
index_mapping = es_client.indices.get_mapping(index=index_name)
print(index_mapping)

{'ml-course-questions': {'mappings': {'properties': {'course': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'id': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'question': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'question_text_vector': {'type': 'float'}, 'question_vector': {'type': 'float'}, 'section': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'text': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'text_vector': {'type': 'float'}}}}}
